# SOEN363 Project

## Drop the required tables **(Only for Debugging Purposes)**

In [ ]:
DROP VIEW high_ratings;

DROP VIEW low_ratings;

DROP TABLE actors;

DROP TABLE genres;

DROP TABLE tags;

DROP TABLE tag_names;

DROP TABLE movies;

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.020

## 2a

In [ ]:
CREATE TABLE movies (
    mid             INTEGER UNIQUE, 
    title           VARCHAR, 
    year            INTERVAL, 
    rating          REAL CHECK (rating BETWEEN 0.0 AND 5.0), 
    num_ratings     INTEGER,
    PRIMARY KEY     (mid, title, year)
);

CREATE TABLE actors (
    mid             INTEGER REFERENCES movies(mid), 
    name            VARCHAR, 
    cast_position   INTEGER,
    PRIMARY KEY     (mid, name)
);

CREATE TABLE genres (
    mid             INTEGER REFERENCES movies(mid), 
    genre           VARCHAR,
    PRIMARY KEY     (mid, genre)
);

CREATE TABLE tag_names (
    tid             INTEGER, 
    tag             VARCHAR,
    PRIMARY KEY     (tid)
);

CREATE TABLE tags (
    mid             INTEGER REFERENCES movies(mid), 
    tid             INTEGER,
    PRIMARY KEY     (mid, tid),
    FOREIGN KEY     (tid) REFERENCES tag_names(tid)
);

COPY movies FROM '/srv/soen363/phase1/movies.dat';

COPY actors FROM '/srv/soen363/phase1/actors.dat';

COPY genres FROM '/srv/soen363/phase1/genres.dat';

COPY tag_names FROM '/srv/soen363/phase1/tag_names.dat';

COPY tags FROM '/srv/soen363/phase1/tags.dat';

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

...

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:02.522

## 3a

## 3b

## 3c

## 3d

## 3e

## 3f

## 3g

## 3h

In [ ]:
-- High Ratings view to get actors the Top 10 that have participated in a movie with a rating >= 4
CREATE OR REPLACE VIEW high_ratings AS
    SELECT DISTINCT A.name 
    FROM actors A, movies M
    WHERE A.mid = m.mid AND m.rating >= 4; 

-- Low Ratings view to g
LIMIT 10;t actors that have participated in a movie with a rating < 4
CREATE OR REPLACE VIEW low_ratings AS
    SELECT DISTINCT A.name 
    FROM actors A, movies M

-- Get the actors count coming from High Ratings 
SELECT COUNT(*) as result_count
FROM high_ratings;
    WHERE A.mid = m.mid AND m.rating < 4;

-- Get the actors count coming from Low Ratings
SELECT COUNT(*) as result_count
FROM low_ratings;

Commands completed successfully

Commands completed successfully

(1 row(s) affected)

(1 row(s) affected)

Total execution time: 00:00:00.132

result_count
13710


result_count
87032


In [ ]:
-- Get the number of No Flop actors in the database
SELECT COUNT(*)
FROM high_ratings
WHERE name NOT IN (
    SELECT name
    FROM low_ratings
);

(1 row(s) affected)

Total execution time: 00:00:00.125

count
7015


In [ ]:
-- Get the name and movie count of No Flop actors
SELECT R.name, COUNT(A.*) as movie_count
FROM high_ratings R, actors A
WHERE 
    R.name NOT IN (
        SELECT name
        FROM low_ratings
    ) AND 
    A.name = R.NAME
GROUP BY R.name
ORDER BY COUNT(A.*) DESC;

(10 row(s) affected)

Total execution time: 00:00:00.170

name,movie_count
Nikolai Grinko,8
John Cazale,7
Paul Frankeur,7
Allan Garcia,6
Gunnel Lindblom,6
Tsutomu Yamazaki,6
Kuniko Miyake,6
Megan Gallagher,5
Anatoli Solonitsin,5
Timothy T. Mitchum,5


## 3i

## 3j

## 3k

## 3l

## 3m

## 4a

## 4b

## 4c

## 4d